## Linguas I - Limpeza de dados

#### Bibliotecas e Base de Dados

In [1]:
import pandas as pd
import os
import math
from statsmodels.tsa.seasonal import STL

%matplotlib inline
import matplotlib.pyplot as plt


In [3]:
## print("Esperamos trabalhar no diretório")
print(os.getcwd())


/Users/liviacarvalho/Documents/Insper Junior/Capacitação/Case I - Linguas I/LinguasI-Analise-Exploratoria


In [4]:
filename = 'base_cliente.xlsx'

if filename in os.listdir():
    print(f'Parece que o arquivo {filename} está na mesma pasta do notebook da avaliação, yay!\n')
    
else:
    print(f'Não encontrei o arquivo {filename}.\n'
          f'O notebook e os arquivos de dados desta avaliação devem ser salvos dentro da mesma pasta no seu computador.\n')
    

Parece que o arquivo base_cliente.xlsx está na mesma pasta do notebook da avaliação, yay!



In [5]:
# Carregando os dados
df = pd.read_excel(filename)
df.head()

,"ID_Cliente,Motivo,Education,Cidade,Renda_anual,Aulas,NPS,Num_Aulas,Ano_de_nascimento,Banheiros_na_casa,Quartos_na_casa"
0,"0,Viagem,GraduaÃ§Ã£o,SÃ£o Paulo,58138.0,Ingles..."
1,"1,Trabalho,GraduaÃ§Ã£o,TaubatÃ©,46344.0,Espanh..."
2,"2,Estudos,graduacao,SÃ£o Bernardo,71613.0,Ingl..."
3,"3,Viagem,graduacao,SÃ£o Paulo,,Espanhol,4,22,2..."
4,"4,Trabalho,Doutorado,SÃ£o Paulo,58293.0,Ingres..."


In [7]:

# Lendo com engine openpyxl
df_raw = pd.read_excel(filename, engine="openpyxl", header=None)

# Separando os dados da única coluna em múltiplas colunas
df = df_raw[0].str.split(",", expand=True)

# Ajustando o nome das colunas (a primeira linha do df agora contém os nomes)
df.columns = df.iloc[0]
df = df[1:]

df.head(100)


,ID_Cliente,Motivo,Education,Cidade,Renda_anual,Aulas,NPS,Num_Aulas,Ano_de_nascimento,Banheiros_na_casa,Quartos_na_casa
1,0,Viagem,GraduaÃ§Ã£o,SÃ£o Paulo,58138.0,Ingles,4,19,1983,3.0,4.0
2,1,Trabalho,GraduaÃ§Ã£o,TaubatÃ©,46344.0,Espanhol,4,10,1980,4.0,3.0
3,2,Estudos,graduacao,SÃ£o Bernardo,71613.0,Ingles,3,59,1991,4.0,5.0
4,3,Viagem,graduacao,SÃ£o Paulo,,Espanhol,4,22,2010,1,1.0
5,4,Trabalho,Doutorado,SÃ£o Paulo,58293.0,Ingres,3,8,2007,3.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...
96,95,Viagem,graduacao,SÃ£o Paulo,,Espanhol,3,19,2009,1,2.0
97,96,Viagem,GraduaÃ§Ã£o,SÃ£o Paulo,47916.0,Espanhol,3,12,1986,3.0,3.0
98,97,Viagem,GraduaÃ§Ã£o,SÃ£o Paulo,51813.0,Ingles,1,15,1998,3.0,4.0
99,98,Estudos,Ensino MÃ©dio,SÃ£o Paulo,78497.0,Espanhol,1,4,1977,4.0,6.0
